In [25]:
# tag means classes
# patterns which means what user is going to ask
# response which is chatbot response

data={"intents":[
    {
        "tag": "greeting",
        "patterns":["Hello", "How are yoy?","Hi There","Hi","What's up with"],
        "responses":["Howdy Partner","Hello","How are you doing?","Gretings!","How do you do?"]
    },
    {
        "tag": "age",
        "patterns":["how old are you", "when is your birthday","when was you born"],
        "responses":["I am 24 years old", "I was born in 1996", "My birthday is July 3rd and I was born in 1996","03/07/1996"]
    },
    {
        "tag": "date",
        "patterns":["what are you doing this wekend",
                   "do you want to hangout sometime?","what are your plans for this week"],
        "responses":["I am available all week","I don't have any plans","I am not busy"]
    },
    {
        "tag": "name",
        "patterns":["what's your name","what are you called","who are you"],
        "responses":["My name is Kippi","I'm Kippi","Kippi"]
    },
    {
        "tag": "goodbye",
        "patterns":["bye","g2g","see ya","adios","cya"],
        "responses":["It was nice speaking to you","See you later","Speak soon"]
    }
]}


In [26]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VIBHANSHU\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VIBHANSHU\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
# initializing lemmatizer to get stem of words

lemmatizer= WordNetLemmatizer()

words=[]
classes=[]
doc_x=[]
doc_y=[]

In [28]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens= nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_x.append(pattern)
        doc_y.append(intent["tag"])
    # add tag to classes
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

In [29]:
# lemmatize all the owrds in the vocab and convert them to lowercase

words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

In [30]:
# sorting the vocab and classes in alphabetical order and taking the set to ensure
# no duplicates occur

words=sorted(set(words))
classes=sorted(set(classes))

In [31]:
# sorting the vocab and classes in alphabetical order and taking the set to ensure
# no duplicates occur

words=sorted(set(words))
classes=sorted(set(classes))

In [32]:
print(words)

["'s", 'adios', 'are', 'birthday', 'born', 'bye', 'called', 'cya', 'do', 'doing', 'for', 'g2g', 'hangout', 'hello', 'hi', 'how', 'is', 'name', 'old', 'plan', 'see', 'sometime', 'there', 'this', 'to', 'up', 'wa', 'want', 'week', 'wekend', 'what', 'when', 'who', 'with', 'ya', 'you', 'your', 'yoy']


In [33]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [34]:
print(doc_x)

['Hello', 'How are yoy?', 'Hi There', 'Hi', "What's up with", 'how old are you', 'when is your birthday', 'when was you born', 'what are you doing this wekend', 'do you want to hangout sometime?', 'what are your plans for this week', "what's your name", 'what are you called', 'who are you', 'bye', 'g2g', 'see ya', 'adios', 'cya']


In [35]:
print(doc_x)


['Hello', 'How are yoy?', 'Hi There', 'Hi', "What's up with", 'how old are you', 'when is your birthday', 'when was you born', 'what are you doing this wekend', 'do you want to hangout sometime?', 'what are your plans for this week', "what's your name", 'what are you called', 'who are you', 'bye', 'g2g', 'see ya', 'adios', 'cya']


In [36]:
# list for training data

training=[]
out_empty=[0]*len(classes)

# create a bag of words model
for idx, doc in enumerate(doc_x):
    bow=[]
    text=lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row= list(out_empty)
    output_row[classes.index(doc_y[idx])]=1
    
    training.append([bow, output_row])
    
random.shuffle(training)

training=np.array(training, dtype=object)


train_X=np.array(list(training[:,0]))
train_y= np.array(list(training[:,1]))

In [37]:
input_shape=(len(train_X[0]),)
output_shape=len(train_y[0])
epochs=200

In [38]:
model=Sequential()
model.add(Dense(128,input_shape=input_shape, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation="softmax"))
adam=tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)

model.compile(loss="categorical_crossentropy",
             optimizer=adam,
             metrics=["accuracy"])

print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               4992      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 13,573
Trainable params: 13,573
Non-trainable params: 0
_________________________________________________________________
None


Epoch 1/200
1/1 [==============================] - 0s 437ms/step - loss: 1.6099 - accuracy: 0.2105
Epoch 2/200
1/1 [==============================] - 0s 8ms/step - loss: 1.5880 - accuracy: 0.2632
Epoch 3/200
1/1 [==============================] - 0s 4ms/step - loss: 1.4629 - accuracy: 0.4211
Epoch 4/200
1/1 [==============================] - 0s 6ms/step - loss: 1.2348 - accuracy: 0.5789
Epoch 5/200
1/1 [==============================] - 0s 6ms/step - loss: 1.1624 - accuracy: 0.6842
Epoch 6/200
1/1 [==============================] - 0s 6ms/step - loss: 1.0000 - accuracy: 0.7895
Epoch 7/200
1/1 [==============================] - 0s 4ms/step - loss: 0.9661 - accuracy: 0.7895
Epoch 8/200
1/1 [==============================] - 0s 4ms/step - loss: 0.7176 - accuracy: 0.9474
Epoch 9/200
1/1 [==============================] - 0s 6ms/step - loss: 0.7680 - accuracy: 0.9474
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 0.4520 - accuracy: 1.0000
Epoch 11/200
1/1 [=========

In [39]:
def clean_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab):
    tokens=clean_text(text)
    bow=[0]*len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word==w:
                bow[idx]=1
    return np.array(bow)

In [40]:
def clean_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab):
    tokens=clean_text(text)
    bow=[0]*len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word==w:
                bow[idx]=1
    return np.array(bow)

In [41]:
def pred_class(text, vocab, labels):
    bow=bag_of_words(text,vocab)
    result= model.predict(np.array([bow]))[0]
    thresh=0.2
    y_pred=[[idx, res] for idx, res in enumerate(result) if res>thresh]
    
    y_pred.sort(key=lambda x:x[1], reverse=True)
    return_list=[]
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    tag=intents_list[0]
    list_of_intents= intents_json["intents"]
    for i in list_of_intents:
        if i["tag"]==tag:
            result= random.choice(i["responses"])
            break
    return result

In [1]:
# running the chatbot

# Assuming you have the functions pred_class and get_response implemented

while True:
    message = input("You: ")

    if message.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break

    # Get the predicted class for the user's message
    intents = pred_class(text, vocab, labels)

    # Get the chatbot's response based on the predicted class
    result = get_response(intents_list, intents_json)

    print("Chatbot:", result)
